In [3]:
import pandas as pd
import numpy as np
import time

import openai    
openai.api_key=''


In [4]:
import os

# list all folder in scrapers
scraper_list = os.listdir('scrapers')

# loop over those folder
for scraper in scraper_list:
    # go inside a folder called result
    result_folder_path = 'scrapers/' + scraper + '/result'

    # list all the files inside result folder
    result_files = os.listdir(result_folder_path)
    print(result_files)

    # loop over those files
    for file in result_files:
        # get the file name 
        file_name = file.split('.')[0]
        print('file name: ', file_name)

        # get the 3 columns from the file: job_title, job_link and description and create a dataframe from them
        file_data = pd.read_csv(result_folder_path + '/' + file)

        

['cloud solutions intern.csv', 'business analyst intern.csv', 'cloud engineer intern.csv', 'cloud infrastructure architect intern.csv', 'cloud architect intern.csv', 'data analyst intern.csv', 'devops engineer intern.csv', 'devops architect intern.csv', 'cloud systems architect intern.csv', 'data intern.csv', 'data scientist intern.csv']
['cloud solutions intern.csv', 'business analyst intern.csv', 'cloud engineer intern.csv', 'cloud infrastructure architect intern.csv', 'cloud architect intern.csv', 'data analyst intern.csv', 'devops engineer intern.csv', 'devops architect intern.csv', 'quantitative analyst intern.csv', 'cloud systems architect intern.csv', 'devops specialist intern.csv', 'data intern.csv', 'data scientist intern.csv', 'data engineer intern.csv']


In [8]:
# list folder
import os
import subprocess

os.listdir('./scrapers/linkedin-scraper/result/')

# get a file in result folder
df = pd.read_csv('./scrapers/linkedin-scraper/result/data scientist intern.csv')

In [9]:
df.head()

,job_title,job_link,description,company_name,company_link,job_location,workplace_type,post_date,number_of_applicants,skill_list,title
0,QA Engineer Intern,https://www.linkedin.com/jobs/view/3637610060/...,About the job\n\n\n\n\nPassionate people. Loya...,SAS,https://www.linkedin.com/company/sas/life/,"Bloomington, MN",On-site,6 hours ago,12 applicants,[],data scientist intern
1,Software Engineering Co-op (Fall 2023),https://www.linkedin.com/jobs/view/3641126197/...,About the job\nJoin a team recognized for lead...,Honeywell,https://www.linkedin.com/company/honeywell/life/,"Alpharetta, GA",On-site,2 hours ago,30 applicants,[],data scientist intern
2,Software Engineering Co-op (Fall 2023),https://www.linkedin.com/jobs/view/3641126196/...,About the job\nJoin a team recognized for lead...,Honeywell,https://www.linkedin.com/company/honeywell/life/,"Mason, OH",On-site,2 hours ago,25 applicants,[],data scientist intern
3,Undergraduate (Year-Round) Intern - PV Perform...,https://www.linkedin.com/jobs/view/3641403578/...,About the job\nPosting Title\n\nUndergraduate ...,National Renewable Energy Laboratory,https://www.linkedin.com/company/national-rene...,"Golden, CO",On-site,17 hours ago,4 applicants,[],data scientist intern
4,2023 Intern - Research Scientist/Engineer,https://www.linkedin.com/jobs/view/3641149301/...,About the job\nOur Company\n\nChanging the wor...,Adobe,https://www.linkedin.com/company/adobe/life/,"San Jose, CA",NaN,50 minutes ago,14 applicants,[],data scientist intern


In [6]:
def get_completion(prompt, model="gpt-3.5-turbo-0613", temperature=0.0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response

In [8]:
response = get_completion(
    prompt="give me the name of 10 animals"
)

In [11]:
response['usage']

<OpenAIObject at 0x7fe41f7da8e0> JSON: {
  "completion_tokens": 45,
  "prompt_tokens": 15,
  "total_tokens": 60
}

In [49]:
def GetSummaryPrompt(job_description):

    prompt = f'''
    Your task is to generate a short summary of the job description to suggest key \ 
    ideas to a job seeker. \

    Summarize the following job description, delimited by tripple bacticks: \
    in at most 3 sentences, and focus on key skills and requirements. \

    Job Description: ```{job_description}```

    '''

    response = get_completion(prompt)
    return response

In [50]:
# Extract key details from the job description
def GetKeyDetails(job_description):

    prompt = f'''
    Your task is to extract key information from a job description. 

    The job description is delimited by tripple bacticks\
    Format your response as a JSON object with the following keys listed below. \
    The description of each key is delimited by double bactick, this information is only for you to understand\
    the content of each key. Don't include it in your response. 
    - "Job Title:" ``What is the job title?``
    - "Technical Skills:" ``Skills like C++, Saleforce, AWS are considered technical skills``
    - "Other Responsibilities:" ``Skills like Business Analytics, Giving data-driven decisions are considered soft skills``
    - "General Technical SKills:" ``Grouping technical skills into a more general category, like "Cloud Computing"``
    - "Background Clearance:" `Broad knowledge of enterprise business systems, but particularly CRM, BI and BPM.
    `Does the job require a background clearance? Answer in Yes or No``
    - "Years of Experience:" ``How many years of experience are required?``
    - "Is this intern or entry position" ``Answer in Yes or No``

    Job Description: ```{job_description}```

    '''

    key_details = get_completion(prompt)
    return key_details

In [51]:
# Some Constant
SLEEP_TIME_OPEN_AI_EACH_ROW = 1
SLEEP_TIME_WHEN_FAILED = 60

In [56]:
# define your own error
class OpenAIError(Exception):
    # define an error called ServiceUnavailableError
    
    pass

In [53]:
def AnalyzeDescription(df):

    # get the number of rows
    num_rows = df.shape[0]

    i = 0
    # loop over each row
    while i < num_rows:
        try:
            # get the job description
            job_description = df.iloc[i]['description']

            # get the job summary
            job_summary = GetSummaryPrompt(job_description)

            # get the key details from the job description
            key_details = GetKeyDetails(job_description)

            # add the job summary and key details to the dataframe
            df.loc[i, 'job_summary'] = job_summary
            df.loc[i, 'key_details'] = key_details

            # increment i
            i += 1

            # sleep for 1 second

        except IndexError as e:
            print(e)
            print('failed at row: ', i)
            time.sleep(SLEEP_TIME_WHEN_FAILED)
            continue

        except Exception as e:
            # this is general error
            print('This is a general error: ', e)
            # no need to go on at this point
            break



In [54]:
new_df = AnalyzeDescription(df)

new_df

KeyboardInterrupt: 

In [61]:
class ServiceUnavailableError(Exception):
    # say that openai is overloaded

    def __init__(self, message="OpenAI is overloaded"):
        self.message = message
        super().__init__(self.message)


In [65]:
for i in range(100):
    try:
        random_job_description = df.iloc[i]['description']

        GetSummaryPrompt(random_job_description)

        GetKeyDetails(random_job_description)
    except IndexError as e:
        print('failed at row: ', i)
        time.sleep(SLEEP_TIME_WHEN_FAILED)

    except ServiceUnavailableError as e:
        print(e)
        print('failed at row: ', i)
        print('open ai fucked up')
        time.sleep(SLEEP_TIME_WHEN_FAILED)

    except Exception as e:
        # general error
        print(e)

failed at row:  18
failed at row:  19
failed at row:  20
failed at row:  21
failed at row:  22
failed at row:  23
failed at row:  24
failed at row:  25
failed at row:  26
failed at row:  27


KeyboardInterrupt: 

In [43]:
new_df.to_csv('new_df.csv', index=False)

## Job Description

In [16]:
print(job_description)

About the job




Passionate people. Loyal clients. Leading solutions.

With a rich culture of creative collaboration and professional growth, IDeaS’ team members build successful careers with us.

IDeaS is proud to be a global powerhouse of innovation and excellence; challenge and reward. No matter where we’re working, our teams come together to create leading revenue management solutions that accelerate our clients’ growth through revenue optimization.

Now we just need you!

As a member of the IDeaS Software Development team our interns will participate in all aspects of the software development lifecycle. You will join an autonomous delivery team that is following agile methodologies in actively creating new features to enhance our revenue management products. While on our team, you will have a great opportunity to expand your technical skillset as well as work in an agile environment that believes in test-first, continuous integration and deployment. Your team’s code will be built

## Get Completion

In [24]:
def get_completion(prompt, model="gpt-3.5-turbo-0613", temperature=0.0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


## Summarize the job description

In [25]:
prompt = f'''
Your task is to generate a short summary of the job description to suggest key \ 
ideas to a job seeker. \

Summarize the following job description, delimited by tripple bacticks: \
in at most 3 sentences, and focus on key skills and requirements. \

Job Description: ```{job_description}```

'''

response = get_completion(prompt)
print(response)

IDeaS is seeking interns to join their Software Development team and participate in all aspects of the software development lifecycle. Key skills and requirements include a planned degree in Computer Science or CE, completed at least 3 years of undergraduate degree, experience with different Test Frameworks/Automation Manual, and the ability to analyze user stories/requirements and design test plans. The company values diversity and strives to create an inclusive environment.


## Job Details

In [29]:
# Extract key details from the job description

prompt = f'''
Your task is to extract key information from a job description. 

The job description is delimited by tripple bacticks\
Format your response as a JSON object with the following keys listed below. \
The description of each key is delimited by double bactick, this information is only for you to understand\
the content of each key. Don't include it in your response. 
- "Job Title:" ``What is the job title?``
- "Technical Skills:" ``Skills like C++, Saleforce, AWS are considered technical skills``
- "Other Responsibilities:" ``Skills like Business Analytics, Giving data-driven decisions are considered soft skills``
- "General Technical SKills:" ``Grouping technical skills into a more general category, like "Cloud Computing"``
- "Background Clearance:" `Broad knowledge of enterprise business systems, but particularly CRM, BI and BPM.
`Does the job require a background clearance? Answer in Yes or No``
- "Years of Experience:" ``How many years of experience are required?``
- "Is this intern or entry position" ``Answer in Yes or No``

Job Description: ```{job_description}```

'''

key_detail_job = get_completion(prompt)
print(key_detail_job)

{
  "Job Title": "Software Development Intern",
  "Technical Skills": [
    "Cypress",
    "SQL Server",
    "MongoDB",
    "Datadog",
    "AWS",
    "API testing",
    "K6 - Performance",
    "RabbitMQ",
    "RestAssured",
    "MySQL",
    "Mobile testing tools IOS/Appium",
    "Selenium"
  ],
  "Other Responsibilities": [
    "Analyze user stories/requirements, design test plan, create and execute test cases.",
    "Enhance continuous integration using Github and other in-house tools.",
    "Under the guidance of the Lead Engineer, design and develop test tools and test strategies.",
    "Resolve test failures due to, but not limited to, integration, cross feature impact."
  ],
  "General Technical Skills": [
    "Software Development",
    "Test Automation",
    "Continuous Integration"
  ],
  "Background Clearance": "No",
  "Years of Experience": "N/A (Internship position)",
  "Is this intern or entry position": "Yes"
}


In [66]:
current_dir = os.getcwd()

# find the scraper folder
scraper_list = os.path.join(current_dir, 'scrapers')
print(scraper_list)

for scraper in os.listdir(scraper_list):
    print(scraper)


/home/thieu/DataScience2/AutoJobSubmittion/run-everyday-stuff/scrapers
